# WOE and SHAP Experiments

Author: https://www.github.com/deburky

In [1]:
from pathlib import Path

import numpy as np
import pandas as pd
from fastwoe import FastWoe
from fisher_scoring import LogisticRegression
from pygam import LinearGAM, s
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import KBinsDiscretizer, PolynomialFeatures

In [44]:
# Set the path to the data directory
data_dir = Path.cwd().parent / "data"

# Load the data
df = pd.read_csv(data_dir / "BankCaseStudyData.csv")

# Define the features and label
features = [
    "Application_Score",
    "Bureau_Score",
    "Number_of_Payments",
]

label = "Final_Decision"

X = df[features]
y = df[label].map({"Accept": 0, "Decline": 1})

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

pipeline = Pipeline(
    [
        (
            "woe",
            FastWoe(
                binning_method="tree",
                tree_kwargs={"max_depth": 2, "min_samples_leaf": 5},
            ),
        ),
        ("logistic_regression", LogisticRegression()),
    ]
)

pipeline.fit(X_train, y_train)
pipeline[-1].display_summary(style="cyan1")

╭── Fisher Scoring Logistic Regression Fit ──╮
│                                            │
│         Total Fisher Scoring Iterations: 8 │
│         Log Likelihood: -3460.4293         │
│         Beta 0 = intercept (bias): True    │
│                                            │
╰────────────────────────────────────────────╯

                          Fisher Scoring Logistic Regression Summary                           
┏━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┓
┃          Parameter ┃ Estimate ┃ Std. Error ┃ Wald Statistic ┃ P-value ┃ Lower CI ┃ Upper CI ┃
┡━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━┩
│   intercept (bias) │ -2.1146  │ 0.0353     │ -59.9044       │ 0.0000  │ -2.1838  │ -2.0454  │
│  Application_Score │ 0.6389   │ 0.0267     │ 23.9613        │ 0.0000  │ 0.5866   │ 0.6911   │
│       Bureau_Score │ 0.4159   │ 0.0261     │ 15.9254        │ 0.0000  │ 0.3647   │ 0.4670   │
│ Number_of_Payments │ 0.8341   │ 0.0457     │ 18.2584        │ 0.0000  │ 0.7446   │ 0.9237   │
└────────────────────┴──────────┴────────────┴────────────────┴─────────┴──────────┴──────────┘

## Experiment 1: Baking Intercept into WOE

WOE is a centered log-odds per bin. Therefore, if we want to fit logistic regression without intercept but still retain calibration, we can bake in the log-odds into the WOE scores.

The transformation is simple:

$$
\text{logit}(p_i) = \text{logit}(p) + \sum_{j=1}^d WOE_{ij}
$$

The we can fit a logistic regression without intercept:

$$
\text{logit}(p_i) = \sum_{j=1}^d \beta_j WOE_{ij}
$$



In [61]:
woe_transformer = FastWoe(
    binning_method="tree",
    tree_kwargs={"max_depth": 2, "min_samples_leaf": 5},
)
woe_transformer.fit(X_train, y_train)

X_train_woe = woe_transformer.transform(X_train)
X_test_woe = woe_transformer.transform(X_test)

log_odds = np.log(y_train.mean()) - np.log(1 - y_train.mean())
print(f"Log odds: {log_odds:.2f}")

# Bake in log odds into WOE scores
X_train_log_odds = X_train_woe + log_odds

# Fit LR
USE_BIAS = False
lr_model_no_bias = LogisticRegression(use_bias=USE_BIAS)
lr_model_no_bias.fit(X_train_log_odds, y_train)
lr_model_no_bias.display_summary(style="cyan1")

# Compare probabilities
y_pred = lr_model_no_bias.predict_proba(X_train_log_odds)[:, 1]
print(round(y_pred.mean(), 4), round(y_train.mean(), 4))

Log odds: -2.15


╭── Fisher Scoring Logistic Regression Fit ──╮
│                                            │
│         Total Fisher Scoring Iterations: 8 │
│         Log Likelihood: -3641.2023         │
│         Beta 0 = intercept (bias): False   │
│                                            │
╰────────────────────────────────────────────╯

                          Fisher Scoring Logistic Regression Summary                           
┏━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┓
┃          Parameter ┃ Estimate ┃ Std. Error ┃ Wald Statistic ┃ P-value ┃ Lower CI ┃ Upper CI ┃
┡━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━┩
│  Application_Score │ 0.5863   │ 0.0273     │ 21.4649        │ 0.0000  │ 0.5328   │ 0.6399   │
│       Bureau_Score │ 0.4196   │ 0.0272     │ 15.4271        │ 0.0000  │ 0.3663   │ 0.4729   │
│ Number_of_Payments │ 0.0358   │ 0.0170     │ 2.1087         │ 0.0350  │ 0.0025   │ 0.0691   │
└────────────────────┴──────────┴────────────┴────────────────┴─────────┴──────────┴──────────┘

0.095 0.1045


## Experiment 2: SHAP Logistic Regression

Here we use SHAP values as features in the logistic regression.

In [62]:
import catboost as cb

cb_model = cb.CatBoostClassifier(
    n_estimators=100,
    learning_rate=0.01,
    max_depth=5,
    allow_writing_files=False,
    verbose=False,
)

train_pool = cb.Pool(X_train, y_train)
test_pool = cb.Pool(X_test)

cb_model.fit(train_pool)

shap_values = cb_model.get_feature_importance(
    type="ShapValues",
    data=train_pool,
)

preds = cb_model.predict_proba(X_test)[:, 1]
gini_score = roc_auc_score(y_test, preds) * 2 - 1
print(f"Gini score CatBoost: {gini_score:.4f}")

# Create SHAP df dropping last column (bias)
shap_values_df = pd.DataFrame(
    shap_values[:, :-1], index=X_train.index, columns=X_train.columns
)

lr_shap = LogisticRegression(use_bias=True)
lr_shap.fit(shap_values_df, y_train)
lr_shap.display_summary(style="cyan1")

y_pred = lr_shap.predict_proba(shap_values_df)[:, 1]
print(round(y_pred.mean(), 4), round(y_train.mean(), 4))

gini_score = roc_auc_score(y_test, preds) * 2 - 1
print(f"Gini score SHAP Logistic Regression: {gini_score:.4f}")

Gini score CatBoost: 0.8964


╭── Fisher Scoring Logistic Regression Fit ──╮
│                                            │
│         Total Fisher Scoring Iterations: 8 │
│         Log Likelihood: -3277.3460         │
│         Beta 0 = intercept (bias): True    │
│                                            │
╰────────────────────────────────────────────╯

                          Fisher Scoring Logistic Regression Summary                           
┏━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┓
┃          Parameter ┃ Estimate ┃ Std. Error ┃ Wald Statistic ┃ P-value ┃ Lower CI ┃ Upper CI ┃
┡━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━┩
│   intercept (bias) │ -3.1487  │ 0.0432     │ -72.9375       │ 0.0000  │ -3.2333  │ -3.0641  │
│  Application_Score │ 2.5851   │ 0.1758     │ 14.7087        │ 0.0000  │ 2.2406   │ 2.9296   │
│       Bureau_Score │ 2.8374   │ 0.1710     │ 16.5966        │ 0.0000  │ 2.5024   │ 3.1725   │
│ Number_of_Payments │ 3.3534   │ 0.1195     │ 28.0668        │ 0.0000  │ 3.1192   │ 3.5875   │
└────────────────────┴──────────┴────────────┴────────────────┴─────────┴──────────┴──────────┘

0.1045 0.1045
Gini score SHAP Logistic Regression: 0.8964


## Experiment 3: SHAP Distillation: Binning vs. GAM

We approximate the **teacher model** (CatBoost) with a simpler **student**  
(Logistic Regression) by mapping raw features into SHAP-derived features.

### Teacher
CatBoost learns a nonlinear function
$$
p_i^{\text{teacher}} = \sigma(f(x_i)),
$$
with SHAP decomposition
$$
f(x_i) \approx \phi_0 + \sum_{j=1}^d \phi_{ij}.
$$

### Student A: SHAP-Binned LR
1. Bin raw features into intervals.  
2. Replace each raw $x_{ij}$ by its average SHAP contribution $\tilde{\phi}_{ij}$.  
3. Optionally add interaction terms $\tilde{\phi}_{ij}\tilde{\phi}_{ik}$.  
4. Fit logistic regression:
$$
\hat{p}_i = \sigma\!\Big(\beta_0 + \sum_j \beta_j \tilde{\phi}_{ij} 
+ \sum_{j<k} \beta_{jk}\tilde{\phi}_{ij}\tilde{\phi}_{ik}\Big).
$$

### Student B: SHAP-GAM LR
1. Fit a **GAM** $g_j$ such that $g_j(x_{ij}) \approx \phi_{ij}$.  
2. Transform raw inputs via GAM predictions.  
3. Expand with polynomial interactions.  
4. Fit logistic regression on GAM-mapped SHAP features.

Both students yield **linear, interpretable scorecards** that approximate  
the teacher while preserving much of its predictive power.


### Student A: SHAP-Binned LR

We distill a **teacher** (CatBoost) into a simpler **student** (Logistic Regression) using SHAP-based features.

CatBoost produces SHAP values
$$
f(x_i) \;\approx\; \phi_0 + \sum_{j=1}^d \phi_{ij},
$$
where $\phi_{ij}$ is the contribution of feature $j$.

### Student
1. **Bin raw features** → map bins to average SHAP contributions $\tilde{\phi}_{ij}$.  
2. Optionally add interaction terms $\tilde{\phi}_{ij}\tilde{\phi}_{ik}$.  

The student is then a logistic regression:
$$
\hat{p}_i = \sigma\!\left(\beta_0 + \sum_j \beta_j \tilde{\phi}_{ij}
+ \sum_{j<k} \beta_{jk}\tilde{\phi}_{ij}\tilde{\phi}_{ik}\right).
$$

This keeps much of the teacher’s predictive power while remaining interpretable and deployable.

In [64]:
def gini_score(y_true, y_prob):
    """Gini score for binary classification."""
    return 2 * roc_auc_score(y_true, y_prob) - 1


def fit_teacher_student(
    X_train, y_train, X_test, y_test, n_bins=20, include_interactions=True
):
    """
    Fit a teacher CatBoost model and a student logistic regression
    distilled via SHAP-binned features (with optional interactions).
    Returns artifacts for deployment.
    """

    # Teacher
    teacher = cb.CatBoostClassifier(
        n_estimators=100,
        learning_rate=0.01,
        max_depth=6,
        allow_writing_files=False,
        verbose=False,
    )
    train_pool = cb.Pool(X_train, y_train)
    teacher.fit(train_pool)

    # SHAP values
    shap_values = teacher.get_feature_importance(type="ShapValues", data=train_pool)
    shap_df = pd.DataFrame(
        shap_values[:, :-1], index=X_train.index, columns=X_train.columns
    )

    # Build raw → SHAP bin mapping
    bin_mappers = {}
    binner = KBinsDiscretizer(n_bins=n_bins, encode="ordinal", strategy="uniform")
    for col in X_train.columns:
        X_binned = binner.fit_transform(X_train[[col]])
        shap_avg = shap_df.groupby(X_binned.flatten())[col].mean()
        bin_mappers[col] = {"edges": binner.bin_edges_[0], "shap": shap_avg.to_dict()}

    def transform(X):
        X_new = pd.DataFrame(index=X.index)
        for col in X.columns:
            edges, shap_map = bin_mappers[col]["edges"], bin_mappers[col]["shap"]
            bins = np.digitize(X[col].values, edges[1:-1], right=False)
            X_new[col] = [shap_map.get(b, 0.0) for b in bins]
        return X_new

    # Transform train/test
    X_train_shap = transform(X_train)
    X_test_shap = transform(X_test)

    poly = None
    if include_interactions:
        poly = PolynomialFeatures(interaction_only=True, include_bias=False)
        X_train_shap = poly.fit_transform(X_train_shap)
        X_test_shap = poly.transform(X_test_shap)

    # Student LR
    lr_student = LogisticRegression()
    lr_student.fit(X_train_shap, y_train)

    # Predictions & Gini
    y_train_pred = lr_student.predict_proba(X_train_shap)[:, 1]
    y_test_pred = lr_student.predict_proba(X_test_shap)[:, 1]

    return {
        "teacher": teacher,
        "lr_student": lr_student,
        "bin_mappers": bin_mappers,
        "poly": poly,
        "transform_func": transform,
        "train_gini": gini_score(y_train, y_train_pred),
        "test_gini": gini_score(y_test, y_test_pred),
    }


def score_new_data(X_new, artifacts):
    """
    Apply trained student model to new raw data.
    """
    transform = artifacts["transform_func"]
    X_new_shap = transform(X_new)

    if artifacts["poly"] is not None:
        X_new_shap = artifacts["poly"].transform(X_new_shap)

    return artifacts["lr_student"].predict_proba(X_new_shap)[:, 1]


# Train student
artifacts = fit_teacher_student(
    X_train, y_train, X_test, y_test, n_bins=30, include_interactions=True
)

print("Train Gini:", round(artifacts["train_gini"], 4))
print("Test Gini :", round(artifacts["test_gini"], 4))

X_new = X_test.sample(100, random_state=42)
y_new = y_test.loc[X_new.index]
y_new_pred = score_new_data(X_new, artifacts)

gini_new = gini_score(y_new, y_new_pred)
print(f"Gini score (new data): {gini_new:.4f}")

Train Gini: 0.8645
Test Gini : 0.8732
Gini score (new data): 0.8800


### Student B: SHAP-GAM LR

We approximate a complex teacher (CatBoost) using a student logistic model built on
SHAP-like features. CatBoost logits decompose as

$$
\text{logit}(p_i) = \phi_0 + \sum_{j=1}^d \phi_{ij}
$$

where $\phi_{ij}$ are SHAP contributions. We fit smooth GAM functions
$g_j(x_{ij}) \approx \phi_{ij}$ to map raw inputs → SHAP-like effects.

To recover lost interactions, we expand these GAM features with pairwise products:

$$
z_i = [g_1(x_{i1}), \dots, g_d(x_{id}), \; g_j(x_{ij}) g_k(x_{ik})]
$$

and train a logistic regression:

$$
\hat{p}_i = \sigma\!\left(\beta_0 + \sum_j \beta_j g_j(x_{ij}) + \sum_{j<k} \beta_{jk} g_j(x_{ij}) g_k(x_{ik})\right)
$$

This student closely matches teacher Gini while remaining interpretable and deployable.

In [66]:
def fit_student_from_shap(
    X_train,
    shap_df,
    X_test,
    y_train,
    y_test,
    n_splines=50,
    include_bias=False,
):
    """Fit a student model from SHAP values using GAM mappings + polynomial expansion."""

    # Fit one GAM per feature
    gam_mappers = {}
    for col in X_train.columns:
        gam = LinearGAM(s(0, n_splines=n_splines)).fit(
            X_train[[col]].values, shap_df[col].values
        )
        gam_mappers[col] = gam

    # Transform with GAM predictions
    def transform_with_gam(X):
        X_new = pd.DataFrame(index=X.index)
        for col in X.columns:
            X_new[col] = gam_mappers[col].predict(X[[col]].values)
        return X_new

    X_train_shap = transform_with_gam(X_train)
    X_test_shap = transform_with_gam(X_test)

    # Polynomial expansion
    poly = PolynomialFeatures(include_bias=include_bias, interaction_only=True)
    X_train_poly = poly.fit_transform(X_train_shap)
    X_test_poly = poly.transform(X_test_shap)

    # Logistic regression on expanded features
    lr_student = LogisticRegression()
    lr_student.fit(X_train_poly, y_train)

    # Predictions & Gini
    y_train_pred = lr_student.predict_proba(X_train_poly)[:, 1]
    y_test_pred = lr_student.predict_proba(X_test_poly)[:, 1]

    return {
        "gam_mappers": gam_mappers,
        "poly": poly,
        "lr_student": lr_student,
        "train_gini": gini_score(y_train, y_train_pred),
        "test_gini": gini_score(y_test, y_test_pred),
    }


def score_new_data_with_shap_gam(X_new, artifacts):
    """
    Apply trained student (GAM + Poly + Logistic Regression) to new raw data.
    """
    # 1. Transform raw X with GAM mappings
    X_new_shap = pd.DataFrame(index=X_new.index)
    for col in X_new.columns:
        X_new_shap[col] = artifacts["gam_mappers"][col].predict(X_new[[col]].values)

    # 2. Apply polynomial feature expansion (if used)
    X_new_poly = artifacts["poly"].transform(X_new_shap)

    # 3. Predict with student logistic regression
    return artifacts["lr_student"].predict_proba(X_new_poly)[:, 1]


# Train teacher
teacher = cb.CatBoostClassifier(
    n_estimators=100,
    learning_rate=0.01,
    max_depth=6,
    allow_writing_files=False,
    verbose=False,
)
teacher.fit(X_train, y_train)

teacher_gini = gini_score(y_train, teacher.predict_proba(X_train)[:, 1])
print(f"Teacher Gini: {teacher_gini:.4f}")

# Compute SHAP values once
shap_values = teacher.get_feature_importance(
    type="ShapValues", data=cb.Pool(X_train, y_train)
)
shap_df = pd.DataFrame(
    shap_values[:, :-1], index=X_train.index, columns=X_train.columns
)

# Run student training with polynomial expansion
results = fit_student_from_shap(
    X_train,
    shap_df,
    X_test,
    y_train,
    y_test,
    n_splines=30,
)

print("Train Gini:", round(results["train_gini"], 4))
print("Test Gini :", round(results["test_gini"], 4))

# Inference on unseen data
X_new = X_test.sample(100, random_state=42)
y_new = y_test.loc[X_new.index]
y_new_pred = score_new_data_with_shap_gam(X_new, results)

gini_new = gini_score(y_new, y_new_pred)
print("Gini score (new data):", round(gini_new, 4))

Teacher Gini: 0.8855
Train Gini: 0.8825
Test Gini : 0.8963
Gini score (new data): 0.88
